In [126]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/axel/TMAIM/ID2223/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [127]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [128]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 21:43:41,572 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 21:43:41,581 INFO: Initializing external client
2025-11-18 21:43:41,581 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 21:43:42,815 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286302


'{"country": "Sweden", "city": "Sundsvall", "street": "K\\u00f6pmansgatan", "aqicn_url": "https://api.waqi.info/feed/@9328", "latitude": "62.39", "longitude": "17.31"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
air_quality_lag_fg = fs.get_feature_group(
    name='air_quality_lag',
    version=2,
)

In [131]:
df = air_quality_lag_fg.read()
df_aq = df[['date', 'pm25']]
df_aq = df_aq.sort_values("date")
df_aq_last_3 = df_aq.tail(3)
lagged_pm25 = []
for index, row in df_aq_last_3.iterrows():
    #print(f"Date: {row['date']}, Value: {row['pm25']}")
    lagged_pm25.append(row['pm25'])
print(lagged_pm25)

df


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.07s) 
[20.0, 22.0, 15.0]


,date,pm25,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d,country,city,street,url
0,2018-11-13 00:00:00+00:00,28.0,31.0,27.0,34.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
1,2018-04-02 00:00:00+00:00,30.0,27.0,24.0,24.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
2,2020-11-21 00:00:00+00:00,17.0,6.0,10.0,18.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
3,2025-04-08 00:00:00+00:00,12.0,8.0,13.0,18.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
4,2022-12-10 00:00:00+00:00,13.0,12.0,19.0,9.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
...,...,...,...,...,...,...,...,...,...
2351,2023-11-26 00:00:00+00:00,6.0,12.0,5.0,6.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
2352,2025-09-12 00:00:00+00:00,48.0,53.0,33.0,29.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
2353,2025-10-11 00:00:00+00:00,12.0,13.0,13.0,20.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328
2354,2021-03-11 00:00:00+00:00,24.0,16.0,25.0,18.0,Sweden,Sundsvall,Köpmansgatan,https://api.waqi.info/feed/@9328


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [132]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,35.0,Sweden,Sundsvall,Köpmansgatan,2025-11-18,https://api.waqi.info/feed/@9328


In [133]:
aq_today_df["pm25_lag_1d"] = lagged_pm25[2]
aq_today_df["pm25_lag_2d"] = lagged_pm25[1]
aq_today_df["pm25_lag_3d"] = lagged_pm25[0]
aq_today_df["pm25_lag_1d"] = aq_today_df["pm25_lag_1d"].astype("float32")
aq_today_df["pm25_lag_2d"] = aq_today_df["pm25_lag_2d"].astype("float32")
aq_today_df["pm25_lag_3d"] = aq_today_df["pm25_lag_3d"].astype("float32")
aq_today_df

,pm25,country,city,street,date,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
0,35.0,Sweden,Sundsvall,Köpmansgatan,2025-11-18,https://api.waqi.info/feed/@9328,15.0,22.0,20.0


In [134]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   pm25         1 non-null      float32       
 1   country      1 non-null      object        
 2   city         1 non-null      object        
 3   street       1 non-null      object        
 4   date         1 non-null      datetime64[ns]
 5   url          1 non-null      object        
 6   pm25_lag_1d  1 non-null      float32       
 7   pm25_lag_2d  1 non-null      float32       
 8   pm25_lag_3d  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [135]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 62.5°N 17.25°E
Elevation 14.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-4.60,0.0,8.587338,326.976135,Sundsvall
1,2025-11-19,-4.70,0.0,5.937272,345.963715,Sundsvall
2,2025-11-20,-3.30,0.0,10.829958,338.552185,Sundsvall
3,2025-11-21,-4.05,0.0,7.491114,305.217682,Sundsvall
4,2025-11-22,2.95,0.0,15.745627,259.460876,Sundsvall
5,2025-11-23,1.60,0.0,6.638072,220.601212,Sundsvall
6,2025-11-24,-0.80,0.0,9.387651,302.471161,Sundsvall


In [136]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [137]:
# Insert new data
air_quality_lag_fg.insert(aq_today_df)

2025-11-18 21:44:41,611 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286302/fs/1265772/fg/1722045


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_lag_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286302/jobs/named/air_quality_lag_2_offline_fg_materialization/executions


(Job('air_quality_lag_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762212
         }
       },
       "result": {
         "observed_value": 35.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T08:44:41.000611Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expe

In [138]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 21:45:00,483 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286302/fs/1265772/fg/1722046


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286302/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 21:45:19,431 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 21:45:25,942 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 21:47:09,366 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 21:47:09,873 INFO: Waiting for log aggregation to finish.
2025-11-18 21:47:18,462 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762214
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T08:45:00.000483Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
